In [129]:

import matplotlib.pyplot as plt 
import datetime
from datetime import date
import numpy as np
import pandas as pd
import yfinance as yf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

4.7.1


# Create Portfolio

In [130]:
portfolio = sorted('MCD SQ AAPL CLX JNJ KO MGM'.split(" "))
portfolio

['AAPL', 'CLX', 'JNJ', 'KO', 'MCD', 'MGM', 'SQ']

In [131]:
#function to retrieve all data
def fetch(ticker):
    def data(ticker):
        return yf.Ticker(ticker).history(period="3y")
    datas = map(data, portfolio)
    return(pd.concat(datas, keys=portfolio, names=['ticker', 'date']))

full_data = fetch(portfolio)
full_data.head()

Open    High     Low   Close    Volume  Dividends  \
ticker date                                                              
AAPL   2017-07-24  144.33  146.11  143.68  145.78  21493200        0.0   
       2017-07-25  145.50  147.45  145.50  146.40  18853900        0.0   
       2017-07-26  146.98  147.54  146.71  147.09  15781000        0.0   
       2017-07-27  147.37  147.60  141.19  144.31  32476300        0.0   
       2017-07-28  143.67  143.99  143.00  143.29  17213700        0.0   

                   Stock Splits  
ticker date                      
AAPL   2017-07-24             0  
       2017-07-25             0  
       2017-07-26             0  
       2017-07-27             0  
       2017-07-28             0

In [132]:
closing = full_data[['Close']].reset_index().pivot("date", "ticker", "Close")
closing.head(10)

ticker,AAPL,CLX,JNJ,KO,MCD,MGM,SQ
date,,,,,,,
2017-07-24,145.78,122.14,122.75,40.61,141.08,32.38,26.75
2017-07-25,146.40,123.58,121.71,40.97,147.78,32.65,26.89
2017-07-26,147.09,124.10,120.85,41.43,145.40,32.26,27.31
2017-07-27,144.31,124.79,120.74,41.77,145.81,31.81,25.92
2017-07-28,143.29,123.48,121.68,41.67,144.79,31.57,26.20
2017-07-31,142.56,124.15,122.48,41.52,144.13,31.37,26.35
2017-08-01,143.82,123.82,122.29,41.39,143.11,30.87,26.81
2017-08-02,150.62,122.74,121.97,41.29,145.48,30.46,26.46
2017-08-03,149.11,125.31,123.06,41.36,143.74,30.06,25.22


In [133]:
purchase_dates =[
    '2017-08-03', '2019-09-04', '2018-10-26','2020-02-03','2019-10-07','2019-01-14','2020-03-16']
purchase_dict = dict(zip(portfolio, purchase_dates))
purchase_dict

{'AAPL': '2017-08-03',
 'CLX': '2019-09-04',
 'JNJ': '2018-10-26',
 'KO': '2020-02-03',
 'MCD': '2019-10-07',
 'MGM': '2019-01-14',
 'SQ': '2020-03-16'}

In [134]:
purchase_prices=[closing.loc[purchase_dict[key],key] for key in purchase_dict]
purchase_prices

[149.11, 161.23, 130.63, 57.55, 207.88, 26.7, 41.27]

In [135]:
df = pd.DataFrame(columns='Ticker Date_Acquired Unit_Cost'.split(' '))
df.Ticker = portfolio
df.Date_Acquired = purchase_dates
df.Unit_Cost = purchase_prices
df['Quantity'] = [8, 11, 5, 8, 6, 15, 26]
df

,Ticker,Date_Acquired,Unit_Cost,Quantity
0,AAPL,2017-08-03,149.11,8
1,CLX,2019-09-04,161.23,11
2,JNJ,2018-10-26,130.63,5
3,KO,2020-02-03,57.55,8
4,MCD,2019-10-07,207.88,6
5,MGM,2019-01-14,26.70,15
6,SQ,2020-03-16,41.27,26


In [136]:
df['Cost_Basis'] = df['Unit_Cost'] *df['Quantity']
df['Current_Price'] = list(closing.iloc[-1])
df

,Ticker,Date_Acquired,Unit_Cost,Quantity,Cost_Basis,Current_Price
0,AAPL,2017-08-03,149.11,8,1192.88,382.40
1,CLX,2019-09-04,161.23,11,1773.53,230.50
2,JNJ,2018-10-26,130.63,5,653.15,150.14
3,KO,2020-02-03,57.55,8,460.40,48.35
4,MCD,2019-10-07,207.88,6,1247.28,198.79
5,MGM,2019-01-14,26.70,15,400.50,15.83
6,SQ,2020-03-16,41.27,26,1073.02,126.76


In [137]:
df['Return'] = round((df['Current_Price']/df['Unit_Cost'])-1,3)
df

,Ticker,Date_Acquired,Unit_Cost,Quantity,Cost_Basis,Current_Price,Return
0,AAPL,2017-08-03,149.11,8,1192.88,382.40,1.565
1,CLX,2019-09-04,161.23,11,1773.53,230.50,0.430
2,JNJ,2018-10-26,130.63,5,653.15,150.14,0.149
3,KO,2020-02-03,57.55,8,460.40,48.35,-0.160
4,MCD,2019-10-07,207.88,6,1247.28,198.79,-0.044
5,MGM,2019-01-14,26.70,15,400.50,15.83,-0.407
6,SQ,2020-03-16,41.27,26,1073.02,126.76,2.071


# Fetch S&P Returns from Each Purchase Period

In [138]:
spy = yf.Ticker("SPY").history(period="3y").Close
spy

Date
2017-07-24    232.94
2017-07-25    233.51
2017-07-26    233.52
2017-07-27    233.30
2017-07-28    233.02
               ...  
2020-07-17    321.72
2020-07-20    324.32
2020-07-21    325.01
2020-07-22    326.86
2020-07-23    326.61
Name: Close, Length: 756, dtype: float64

In [139]:
def get_growth_rate(purchase_date):
    rate = (spy[-1]/spy.loc[purchase_date])-1
    return round(rate, 3)

In [140]:
df['SPY_Growth'] = [get_growth_rate(date) for date in df['Date_Acquired']]
df

,Ticker,Date_Acquired,Unit_Cost,Quantity,Cost_Basis,Current_Price,Return,SPY_Growth
0,AAPL,2017-08-03,149.11,8,1192.88,382.40,1.565,0.401
1,CLX,2019-09-04,161.23,11,1773.53,230.50,0.430,0.133
2,JNJ,2018-10-26,130.63,5,653.15,150.14,0.149,0.275
3,KO,2020-02-03,57.55,8,460.40,48.35,-0.160,0.018
4,MCD,2019-10-07,207.88,6,1247.28,198.79,-0.044,0.131
5,MGM,2019-01-14,26.70,15,400.50,15.83,-0.407,0.306
6,SQ,2020-03-16,41.27,26,1073.02,126.76,2.071,0.376


In [141]:
df['PnL'] = (df.Current_Price*df.Quantity)-df.Cost_Basis
df['SPY_Return'] = df.Cost_Basis*df.SPY_Growth
df['Return_Diff'] = df.Return - df.SPY_Growth
df['Return_Over_SPY'] = df.Return_Diff*df.Cost_Basis
df

,Ticker,Date_Acquired,Unit_Cost,Quantity,Cost_Basis,Current_Price,Return,SPY_Growth,PnL,SPY_Return,Return_Diff,Return_Over_SPY
0,AAPL,2017-08-03,149.11,8,1192.88,382.40,1.565,0.401,1866.32,478.34488,1.164,1388.51232
1,CLX,2019-09-04,161.23,11,1773.53,230.50,0.430,0.133,761.97,235.87949,0.297,526.73841
2,JNJ,2018-10-26,130.63,5,653.15,150.14,0.149,0.275,97.55,179.61625,-0.126,-82.29690
3,KO,2020-02-03,57.55,8,460.40,48.35,-0.160,0.018,-73.60,8.28720,-0.178,-81.95120
4,MCD,2019-10-07,207.88,6,1247.28,198.79,-0.044,0.131,-54.54,163.39368,-0.175,-218.27400
5,MGM,2019-01-14,26.70,15,400.50,15.83,-0.407,0.306,-163.05,122.55300,-0.713,-285.55650
6,SQ,2020-03-16,41.27,26,1073.02,126.76,2.071,0.376,2222.74,403.45552,1.695,1818.76890


# Visualize

In [142]:
trace1 =go.Bar(
x=df.Ticker, 
y=df.PnL, 
name="PnL")

trace2 = go.Bar(
x=df.Ticker,
y=df.SPY_Return, 
name="S&P Gains"
)

data=[trace1, trace2]

layout = go.Layout(title="PnL vs S&P 500",
                   yaxis=dict(title='Profit and Loss', 
                             tickformat='$')
                  )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [143]:
data=[go.Bar(
x=df.Ticker, 
y=df.Return_Diff)]

layout = go.Layout(title="Returns Beyond S&P 500",
                   yaxis=dict(title='Returns', 
                             tickformat='%')
                  )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

